In [4]:
from datetime import datetime
from typing import Annotated, Literal

from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langgraph.managed import IsLastStep
from langgraph.prebuilt import create_react_agent

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Today is {today}"),
        ("placeholder", "{messages}"),
    ]
)


def check_weather(location: str) -> str:
    """Return the weather forecast for the specified location."""

    return f"It's always sunny in {location}"


tools = [check_weather]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")


class CustomState(TypedDict):
    today: str
    messages: Annotated[list[BaseMessage], add_messages]
    is_last_step: IsLastStep
    remaining_steps: int


graph = create_react_agent(model, tools, state_schema=CustomState, prompt=prompt)
inputs = {
    "messages": [
        ("user", "What's today's date? And what's the weather in San Francisco?")
    ],
    "today": "July 16, 2004",
}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]

    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What's today's date? And what's the weather in San Francisco?
================================== Ai Message ==================================

Today is July 16, 2004. I can check the weather in San Francisco for you. Could you please provide the location?


In [ ]:
graph.invoke(inputs)

{'today': 'July 16, 2004',
 'messages': [HumanMessage(content="What's today's date? And what's the weather in SF?", additional_kwargs={}, response_metadata={}, id='710efb25-7860-4d76-919f-d83cdda0347b'),
  AIMessage(content='Today is July 16, 2004. I can check the weather in SF if you provide the location.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b7d24fbd-b48e-430b-a076-9d4abb39166b-0', usage_metadata={'input_tokens': 43, 'output_tokens': 27, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})]}